In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# #import what we need
# from kaggle.competitions import nflrush
# env = nflrush.make_env()

# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split, KFold

# from keras.layers import Dense, Dropout, BatchNormalization
# from keras.models import Sequential
# from keras.callbacks import Callback, EarlyStopping

In [ ]:
def pre_process(train_df):
    
    #填补缺失值
    train_df.loc[train_df.S.isnull(),'S'] = 2.6
    train_df.loc[train_df.A.isnull(),'A'] = 1.6    

In [ ]:
def pre_process(train_df):    
    #初始 跟踪处理 参考https://www.kaggle.com/statsbymichaellopez/nfl-tracking-wrangling-voronoi-and-sonars
    
    #add a moving left flag and get rusher
    train_df['ToLeft'] = train_df['PlayDirection'] == 'left'
    train_df['IsBallCarrier'] = train_df['NflId'] == train_df['NflIdRusher']

In [ ]:
def pre_process(train_df):    
    #修改队伍名拼写错误
    train_df.loc[train_df['VisitorTeamAbbr'] == "ARI", 'VisitorTeamAbbr'] = 'ARZ'
    train_df.loc[train_df['HomeTeamAbbr'] == "ARI", 'HomeTeamAbbr'] = 'ARZ'

    train_df.loc[train_df['VisitorTeamAbbr'] == "BAL", 'VisitorTeamAbbr'] = 'BLT'
    train_df.loc[train_df['HomeTeamAbbr'] == "BAL", 'HomeTeamAbbr'] = 'BLT'

    train_df.loc[train_df['VisitorTeamAbbr'] == "CLE", 'VisitorTeamAbbr'] = 'CLV'
    train_df.loc[train_df['HomeTeamAbbr'] == "CLE", 'HomeTeamAbbr'] = 'CLV'

    train_df.loc[train_df['VisitorTeamAbbr'] == "HOU", 'VisitorTeamAbbr'] = 'HST'
    train_df.loc[train_df['HomeTeamAbbr'] == "HOU", 'HomeTeamAbbr'] = 'HST'

In [ ]:
def pre_process(train_df):    
    #找出进攻团队
    train_df['TeamOnOffense'] = np.where(train_df['PossessionTeam'] == train_df['HomeTeamAbbr'], 'home', 'away')
    train_df['IsOnOffense'] = train_df['TeamOnOffense'] == train_df['Team']

In [ ]:
def pre_process(train_df):    
    #标准化
    train_df['YardsFromOwnGoal'] = np.where(train_df['FieldPosition'] == train_df['PossessionTeam'], train_df['YardLine'], 50 + (50 - train_df['YardLine']))
    train_df['YardsFromOwnGoal'] = np.where(train_df['YardLine'] == 50, 50, train_df['YardsFromOwnGoal'])

In [ ]:
def pre_process(train_df):    
    #标准化X,Y坐标
    train_df['X_std'] = np.where(train_df['ToLeft'], 120 - train_df['X'], train_df['X']) - 10
    train_df['Y_std'] = np.where(train_df['ToLeft'], 160 / 3  - train_df['Y'], train_df['Y'])

In [ ]:
def pre_process(train_df):    
    #标准化方向
    train_df['Dir_std_1'] = np.where( (train_df['Dir'] < 90) & train_df['ToLeft'], train_df['Dir'] + 360, train_df['Dir'])
    train_df['Dir_std_1'] = np.where( (train_df['Dir'] > 270) & ~train_df['ToLeft'], train_df['Dir'] - 360, train_df['Dir_std_1'])

    train_df['Dir_std_2'] = np.where(train_df['ToLeft'], train_df['Dir_std_1'] - 180, train_df['Dir_std_1'])
    
    train_df['Dir_std_2'] = train_df['Dir_std_2']*np.pi/180

In [ ]:
def pre_process(train_df):    
    #fill any na's in the standardized direction
    train_df.loc[train_df.Dir_std_2.isnull(),'Dir_std_2'] = np.pi/2

In [ ]:
def pre_process(train_df):    
    #Dir_std_2 goes from -90 to 270    
    def x_angle_mult(row):
        if row.Dir_std_2 <= 0 and row.Dir_std_2 >= -np.pi/2:
            return -1 * np.sin( abs(row.Dir_std_2) )
        if row.Dir_std_2 > 0 and row.Dir_std_2 <= np.pi/2:
            return np.sin(row.Dir_std_2)
        if row.Dir_std_2 > np.pi/2 and row.Dir_std_2 <= np.pi:
            return np.sin(np.pi - row.Dir_std_2)
        if row.Dir_std_2 > np.pi and row.Dir_std_2 <= 1.5*np.pi:
            return -1 * np.sin(row.Dir_std_2 - np.pi)
        return 1
    
    def y_angle_mult(row):
        if row.Dir_std_2 <= 0 and row.Dir_std_2 >= -np.pi/2:
            return np.cos( abs(row.Dir_std_2) )
        if row.Dir_std_2 > 0 and row.Dir_std_2 <= np.pi/2:
            return np.cos(row.Dir_std_2)
        if row.Dir_std_2 > np.pi/2 and row.Dir_std_2 <= np.pi:
            return -1 * np.cos(np.pi - row.Dir_std_2)
        if row.Dir_std_2 > np.pi and row.Dir_std_2 <= 1.5*np.pi:
            return -1 * np.cos(row.Dir_std_2 - np.pi)
        return 1
    
    train_df['X_Angle_Mult'] = train_df.apply(x_angle_mult, axis = 1)
    train_df['Y_Angle_Mult'] = train_df.apply(y_angle_mult, axis = 1)

    train_df['X_Speed'] = train_df['S'] * train_df['X_Angle_Mult']
    train_df['Y_Speed'] = train_df['S'] * train_df['Y_Angle_Mult']
    
    train_df['X_Acc'] = train_df['A'] * train_df['X_Angle_Mult']
    train_df['Y_Acc'] = train_df['A'] * train_df['Y_Angle_Mult']
    
    train_df['X_std_new'] = train_df['X_Speed'] + train_df['X_std']
    train_df['Y_std_new'] = train_df['Y_Speed'] + train_df['Y_std']
    
    train_df['X_std_half_new'] = 0.5*train_df['X_Speed'] + train_df['X_std']
    train_df['Y_std_half_new'] = 0.5*train_df['Y_Speed'] + train_df['Y_std']

In [ ]:
def pre_process(train_df):    
    # drop columns we don't need
    drop_cols = ['GameId','Team','X','Y','Dis','Orientation','Dir','DisplayName','JerseyNumber','Season','YardLine','Quarter'
                 ,'GameClock','PossessionTeam','Down','FieldPosition','HomeScoreBeforePlay','VisitorScoreBeforePlay','NflIdRusher',
                 'OffensePersonnel','PlayDirection','TimeHandoff','TimeSnap','PlayerBirthDate','PlayerCollegeName','HomeTeamAbbr','VisitorTeamAbbr',
                'Week','Stadium','Location','StadiumType','Turf','ToLeft','WindDirection','WindSpeed','TeamOnOffense','Dir_std_1']

    train_df.drop(drop_cols, axis = 1, inplace = True)

In [ ]:
def pre_process(train_df):    
    # work on the play_info df
    play_info_cols = ['PlayId','Distance', 'DefendersInTheBox','YardsFromOwnGoal']

    play_train_df = train_df[play_info_cols]

In [ ]:
def pre_process(train_df):    
    #drop dupes
    play_train_df = play_train_df.drop_duplicates()  

In [ ]:
def pre_process(train_df):    
    # fill some na's with average value
    play_train_df.loc[play_train_df.DefendersInTheBox.isnull(), 'DefendersInTheBox'] = 7.0
    play_train_df.loc[play_train_df.Distance.isnull(), 'Distance'] = 8.3 
    play_train_df.loc[play_train_df.YardsFromOwnGoal.isnull(), 'YardsFromOwnGoal'] = 50  

In [ ]:
def pre_process(train_df):    
    #get rusher to game_info
    rushers = train_df.loc[train_df.IsBallCarrier,['PlayId','X_std','Y_std','X_std_new','Y_std_new','X_std_half_new','Y_std_half_new','S','A','X_Speed','Y_Speed','X_Acc','Y_Acc','Dir_std_2']]
    rushers.columns = ['PlayId','X_std_rush','Y_std_rush','X_std_new_rush','Y_std_new_rush','X_std_half_new_rush','Y_std_half_new_rush','S_Rush','A_Rush','X_Speed_Rush','Y_Speed_Rush',
                       'X_Acc_Rush','Y_Acc_Rush','Dir_std_2_Rush']
       
    play_train_df = play_train_df.merge(rushers,how = 'left', on = 'PlayId' )
    play_train_df['DistToLOS_Rusher'] = play_train_df['X_std_rush'] - play_train_df['YardsFromOwnGoal']

    train_df = train_df.merge(rushers[['PlayId','X_std_rush','Y_std_rush','X_std_new_rush','Y_std_new_rush','X_std_half_new_rush','Y_std_half_new_rush','Dir_std_2_Rush']], how = 'left', on = 'PlayId')

    train_df['X_rel_rush'] = train_df['X_std_rush'] - train_df['X_std']
    train_df['Y_rel_rush'] = train_df['Y_std_rush'] - train_df['Y_std']
    
    train_df['X_rel_rush_new'] = train_df['X_std_new_rush'] - train_df['X_std_new']
    train_df['Y_rel_rush_new'] = train_df['Y_std_new_rush'] - train_df['Y_std_new']
    
    train_df['X_rel_rush_half_new'] = train_df['X_std_half_new_rush'] - train_df['X_std_half_new']
    train_df['Y_rel_rush_half_new'] = train_df['Y_std_half_new_rush'] - train_df['Y_std_half_new']

    train_df['DistToRusher'] = np.sqrt( train_df['X_rel_rush']**2 + train_df['Y_rel_rush']**2 )
    train_df['DistToRusherDef'] = train_df.apply(lambda row: row.DistToRusher if not row.IsOnOffense else np.NaN, axis = 1)
    
    train_df['DistToRusherNew'] = np.sqrt( train_df['X_rel_rush_new']**2 + train_df['Y_rel_rush_new']**2 )
    train_df['DistToRusherDefNew'] = train_df.apply(lambda row: row.DistToRusherNew if not row.IsOnOffense else np.NaN, axis = 1)
    
    train_df['DistToRusherHalfNew'] = np.sqrt( train_df['X_rel_rush_half_new']**2 + train_df['Y_rel_rush_half_new']**2 )
    train_df['DistToRusherDefHalfNew'] = train_df.apply(lambda row: row.DistToRusherHalfNew if not row.IsOnOffense else np.NaN, axis = 1)

In [ ]:
def pre_process(train_df):        
    ##
    mean_def_dist   = train_df[['PlayId', 'DistToRusherDef']].groupby('PlayId').agg(DistToRusherDefMean = ('DistToRusherDef',np.nanmean))
    min_def_dist    = train_df[['PlayId', 'DistToRusherDef']].groupby('PlayId').agg(DistToRusherDefMin = ('DistToRusherDef',np.nanmin))
    
    mean_def_dist_new   = train_df[['PlayId', 'DistToRusherDefNew']].groupby('PlayId').agg(DistToRusherDefMeanNew = ('DistToRusherDefNew',np.nanmean))
    min_def_dist_new    = train_df[['PlayId', 'DistToRusherDefNew']].groupby('PlayId').agg(DistToRusherDefMinNew = ('DistToRusherDefNew',np.nanmin))
    
    mean_def_dist_half_new   = train_df[['PlayId', 'DistToRusherDefHalfNew']].groupby('PlayId').agg(DistToRusherDefMeanHalfNew = ('DistToRusherDefHalfNew',np.nanmean))
    min_def_dist_half_new    = train_df[['PlayId', 'DistToRusherDefHalfNew']].groupby('PlayId').agg(DistToRusherDefMinHalfNew = ('DistToRusherDefHalfNew',np.nanmin))
    
    play_train_df = play_train_df.merge(mean_def_dist, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(min_def_dist, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(mean_def_dist_new, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(min_def_dist_new, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(mean_def_dist_half_new, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(min_def_dist_half_new, how = 'left', on = 'PlayId')
    
    train_df.loc[train_df.DistToRusherDef.isnull(),'DistToRusherDef'] = 0
    train_df.loc[train_df.DistToRusherDefNew.isnull(),'DistToRusherDefNew'] = 0
    train_df.loc[train_df.DistToRusherDefHalfNew.isnull(),'DistToRusherDefHalfNew'] = 0
     
    
    def add_boxes(row, xcol, ycol):        
           
        if (row[ycol] >= 2.5) and (row[ycol] <= 7.5) and (row[xcol] <= 0) and (row[xcol] >= -5):
            return 1
        if (row[ycol] >= 2.5) and (row[ycol]<= 7.5) and (row[xcol] <= -5) and (row[xcol] >= -10):
            return 2
        if (row[ycol] >= 2.5) and (row[ycol] <= 7.5) and (row[xcol] <= -10) and (row[xcol] >= -15):
            return 3
        if ( abs(row[ycol]) <= 2.5) and (row[xcol] <= 0) and (row[xcol] >= -5):
            return 4
        if ( abs(row[ycol]) <= 2.5)  and (row[xcol] <= -5) and (row[xcol] >= -10):
            return 5
        if ( abs(row[ycol]) <= 2.5)  and (row[xcol] <= -10) and (row[xcol] >= -15):
            return 6
        if (row[ycol] <= -2.5) and (row[ycol] >= -7.5) and (row[xcol] <= 0) and (row[xcol]>= -5):
            return 7
        if (row[ycol] <= -2.5) and (row[ycol] >= -7.5) and (row[xcol] <= -5) and (row[xcol] >= -10):
            return 8
        if (row[ycol] <= -2.5) and (row[ycol] >= -7.5) and (row[xcol] <= -10) and (row[xcol] >= -15):
            return 9
        return np.NaN
        
        
    train_df['Box'] = train_df.apply(add_boxes, axis = 1,xcol = 'X_rel_rush', ycol = 'Y_rel_rush')
    train_df['BoxNew'] = train_df.apply(add_boxes, axis = 1,xcol = 'X_rel_rush_new', ycol = 'Y_rel_rush_new')
    train_df['BoxHalfNew'] = train_df.apply(add_boxes, axis = 1,xcol = 'X_rel_rush_half_new', ycol = 'Y_rel_rush_half_new')

In [ ]:
def pre_process(train_df):        
    #defenders in box
    defs_in_b1 = train_df.loc[(~train_df.IsOnOffense) & (train_df.Box == 1),['PlayId','Box']].groupby('PlayId').count()
    defs_in_b2 = train_df.loc[(~train_df.IsOnOffense) & (train_df.Box == 2),['PlayId','Box']].groupby('PlayId').count()
    defs_in_b3 = train_df.loc[(~train_df.IsOnOffense) & (train_df.Box == 3),['PlayId','Box']].groupby('PlayId').count()
    defs_in_b4 = train_df.loc[(~train_df.IsOnOffense) & (train_df.Box == 4),['PlayId','Box']].groupby('PlayId').count()
    defs_in_b5 = train_df.loc[(~train_df.IsOnOffense) & (train_df.Box == 5),['PlayId','Box']].groupby('PlayId').count()
    defs_in_b6 = train_df.loc[(~train_df.IsOnOffense) & (train_df.Box == 6),['PlayId','Box']].groupby('PlayId').count()
    defs_in_b7 = train_df.loc[(~train_df.IsOnOffense) & (train_df.Box == 7),['PlayId','Box']].groupby('PlayId').count()
    defs_in_b8 = train_df.loc[(~train_df.IsOnOffense) & (train_df.Box == 8),['PlayId','Box']].groupby('PlayId').count()
    defs_in_b9 = train_df.loc[(~train_df.IsOnOffense) & (train_df.Box == 9),['PlayId','Box']].groupby('PlayId').count()
    
    defs_in_bcols = ['DefsInb1','DefsInb2','DefsInb3','DefsInb4','DefsInb5','DefsInb6','DefsInb7','DefsInb8','DefsInb9']
    
    defs_in_b1.columns = [defs_in_bcols[0]]
    defs_in_b2.columns = [defs_in_bcols[1]]
    defs_in_b3.columns = [defs_in_bcols[2]]
    defs_in_b4.columns = [defs_in_bcols[3]]
    defs_in_b5.columns = [defs_in_bcols[4]]
    defs_in_b6.columns = [defs_in_bcols[5]]
    defs_in_b7.columns = [defs_in_bcols[6]]
    defs_in_b8.columns = [defs_in_bcols[7]]
    defs_in_b9.columns = [defs_in_bcols[8]]
    
    play_train_df = play_train_df.merge(defs_in_b1, how = 'left', on = 'PlayId') 
    play_train_df = play_train_df.merge(defs_in_b2, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_b3, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_b4, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_b5, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_b6, how = 'left', on = 'PlayId') 
    play_train_df = play_train_df.merge(defs_in_b7, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_b8, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_b9, how = 'left', on = 'PlayId')

In [ ]:
def pre_process(train_df):        
    
    #defenders in box HALF NEW
    defs_in_b1HN = train_df.loc[(~train_df.IsOnOffense) & (train_df.BoxHalfNew == 1),['PlayId','BoxHalfNew']].groupby('PlayId').count()
    defs_in_b2HN = train_df.loc[(~train_df.IsOnOffense) & (train_df.BoxHalfNew == 2),['PlayId','BoxHalfNew']].groupby('PlayId').count()
    defs_in_b3HN = train_df.loc[(~train_df.IsOnOffense) & (train_df.BoxHalfNew == 3),['PlayId','BoxHalfNew']].groupby('PlayId').count()
    defs_in_b4HN = train_df.loc[(~train_df.IsOnOffense) & (train_df.BoxHalfNew == 4),['PlayId','BoxHalfNew']].groupby('PlayId').count()
    defs_in_b5HN = train_df.loc[(~train_df.IsOnOffense) & (train_df.BoxHalfNew == 5),['PlayId','BoxHalfNew']].groupby('PlayId').count()
    defs_in_b6HN = train_df.loc[(~train_df.IsOnOffense) & (train_df.BoxHalfNew == 6),['PlayId','BoxHalfNew']].groupby('PlayId').count()
    defs_in_b7HN = train_df.loc[(~train_df.IsOnOffense) & (train_df.BoxHalfNew == 7),['PlayId','BoxHalfNew']].groupby('PlayId').count()
    defs_in_b8HN = train_df.loc[(~train_df.IsOnOffense) & (train_df.BoxHalfNew == 8),['PlayId','BoxHalfNew']].groupby('PlayId').count()
    defs_in_b9HN = train_df.loc[(~train_df.IsOnOffense) & (train_df.BoxHalfNew == 9),['PlayId','BoxHalfNew']].groupby('PlayId').count()
    
    defs_in_bcolsHN = ['DefsInb1HN','DefsInb2HN','DefsInb3HN','DefsInb4HN','DefsInb5HN','DefsInb6HN','DefsInb7HN','DefsInb8HN','DefsInb9HN']
    
    defs_in_b1HN.columns = [defs_in_bcolsHN[0]]
    defs_in_b2HN.columns = [defs_in_bcolsHN[1]]
    defs_in_b3HN.columns = [defs_in_bcolsHN[2]]
    defs_in_b4HN.columns = [defs_in_bcolsHN[3]]
    defs_in_b5HN.columns = [defs_in_bcolsHN[4]]
    defs_in_b6HN.columns = [defs_in_bcolsHN[5]]
    defs_in_b7HN.columns = [defs_in_bcolsHN[6]]
    defs_in_b8HN.columns = [defs_in_bcolsHN[7]]
    defs_in_b9HN.columns = [defs_in_bcolsHN[8]]
    
    play_train_df = play_train_df.merge(defs_in_b1HN, how = 'left', on = 'PlayId') 
    play_train_df = play_train_df.merge(defs_in_b2HN, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_b3HN, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_b4HN, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_b5HN, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_b6HN, how = 'left', on = 'PlayId') 
    play_train_df = play_train_df.merge(defs_in_b7HN, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_b8HN, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_b9HN, how = 'left', on = 'PlayId')

In [ ]:
def pre_process(train_df):        
    #defenders in box NEW
    defs_in_b1N = train_df.loc[(~train_df.IsOnOffense) & (train_df.BoxNew == 1),['PlayId','BoxNew']].groupby('PlayId').count()
    defs_in_b2N = train_df.loc[(~train_df.IsOnOffense) & (train_df.BoxNew == 2),['PlayId','BoxNew']].groupby('PlayId').count()
    defs_in_b3N = train_df.loc[(~train_df.IsOnOffense) & (train_df.BoxNew == 3),['PlayId','BoxNew']].groupby('PlayId').count()
    defs_in_b4N = train_df.loc[(~train_df.IsOnOffense) & (train_df.BoxNew == 4),['PlayId','BoxNew']].groupby('PlayId').count()
    defs_in_b5N = train_df.loc[(~train_df.IsOnOffense) & (train_df.BoxNew == 5),['PlayId','BoxNew']].groupby('PlayId').count()
    defs_in_b6N = train_df.loc[(~train_df.IsOnOffense) & (train_df.BoxNew == 6),['PlayId','BoxNew']].groupby('PlayId').count()
    defs_in_b7N = train_df.loc[(~train_df.IsOnOffense) & (train_df.BoxNew == 7),['PlayId','BoxNew']].groupby('PlayId').count()
    defs_in_b8N = train_df.loc[(~train_df.IsOnOffense) & (train_df.BoxNew == 8),['PlayId','BoxNew']].groupby('PlayId').count()
    defs_in_b9N = train_df.loc[(~train_df.IsOnOffense) & (train_df.BoxNew == 9),['PlayId','BoxNew']].groupby('PlayId').count()
    
    defs_in_bcolsN = ['DefsInb1N','DefsInb2N','DefsInb3N','DefsInb4N','DefsInb5N','DefsInb6N','DefsInb7N','DefsInb8N','DefsInb9N']
    
    defs_in_b1N.columns = [defs_in_bcolsN[0]]
    defs_in_b2N.columns = [defs_in_bcolsN[1]]
    defs_in_b3N.columns = [defs_in_bcolsN[2]]
    defs_in_b4N.columns = [defs_in_bcolsN[3]]
    defs_in_b5N.columns = [defs_in_bcolsN[4]]
    defs_in_b6N.columns = [defs_in_bcolsN[5]]
    defs_in_b7N.columns = [defs_in_bcolsN[6]]
    defs_in_b8N.columns = [defs_in_bcolsN[7]]
    defs_in_b9N.columns = [defs_in_bcolsN[8]]
    
    play_train_df = play_train_df.merge(defs_in_b1N, how = 'left', on = 'PlayId') 
    play_train_df = play_train_df.merge(defs_in_b2N, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_b3N, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_b4N, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_b5N, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_b6N, how = 'left', on = 'PlayId') 
    play_train_df = play_train_df.merge(defs_in_b7N, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_b8N, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_b9N, how = 'left', on = 'PlayId')    

In [ ]:
def pre_process(train_df):        
    # how many defenders within distances
    defs_in_1 = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDef < 1),['PlayId','DistToRusherDef'] ].groupby('PlayId').count()
    defs_in_2 = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDef < 2),['PlayId','DistToRusherDef'] ].groupby('PlayId').count()
    defs_in_3 = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDef < 3),['PlayId','DistToRusherDef'] ].groupby('PlayId').count()
    defs_in_4 = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDef < 4),['PlayId','DistToRusherDef'] ].groupby('PlayId').count()
    defs_in_5 = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDef < 5),['PlayId','DistToRusherDef'] ].groupby('PlayId').count()
    defs_in_6 = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDef < 6),['PlayId','DistToRusherDef'] ].groupby('PlayId').count()
    defs_in_7 = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDef < 7),['PlayId','DistToRusherDef'] ].groupby('PlayId').count()
    defs_in_8 = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDef < 8),['PlayId','DistToRusherDef'] ].groupby('PlayId').count()
    defs_in_9 = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDef < 9),['PlayId','DistToRusherDef'] ].groupby('PlayId').count()
    defs_in_10 = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDef < 10),['PlayId','DistToRusherDef'] ].groupby('PlayId').count()

    defs_in_cols = ['DefsIn1','DefsIn2','DefsIn3','DefsIn4','DefsIn5','DefsIn6','DefsIn7','DefsIn8','DefsIn9','DefsIn10']

    defs_in_1.columns = [defs_in_cols[0]]
    defs_in_2.columns = [defs_in_cols[1]]
    defs_in_3.columns = [defs_in_cols[2]]
    defs_in_4.columns = [defs_in_cols[3]]
    defs_in_5.columns = [defs_in_cols[4]]
    defs_in_6.columns = [defs_in_cols[5]]
    defs_in_7.columns = [defs_in_cols[6]]
    defs_in_8.columns = [defs_in_cols[7]]
    defs_in_9.columns = [defs_in_cols[8]]
    defs_in_10.columns = [defs_in_cols[9]]

    play_train_df = play_train_df.merge(defs_in_1, how = 'left', on = 'PlayId') 
    play_train_df = play_train_df.merge(defs_in_2, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_3, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_4, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_5, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_6, how = 'left', on = 'PlayId') 
    play_train_df = play_train_df.merge(defs_in_7, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_8, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_9, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_10, how = 'left', on = 'PlayId')
    

In [ ]:
def pre_process(train_df):         
    # how many defenders within distances HALF NEW
    defs_in_1HN = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDefHalfNew < 1),['PlayId','DistToRusherDefHalfNew'] ].groupby('PlayId').count()
    defs_in_2HN = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDefHalfNew < 2),['PlayId','DistToRusherDefHalfNew'] ].groupby('PlayId').count()
    defs_in_3HN = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDefHalfNew < 3),['PlayId','DistToRusherDefHalfNew'] ].groupby('PlayId').count()
    defs_in_4HN = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDefHalfNew < 4),['PlayId','DistToRusherDefHalfNew'] ].groupby('PlayId').count()
    defs_in_5HN = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDefHalfNew < 5),['PlayId','DistToRusherDefHalfNew'] ].groupby('PlayId').count()
    defs_in_6HN = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDefHalfNew < 6),['PlayId','DistToRusherDefHalfNew'] ].groupby('PlayId').count()
    defs_in_7HN = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDefHalfNew < 7),['PlayId','DistToRusherDefHalfNew'] ].groupby('PlayId').count()
    defs_in_8HN = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDefHalfNew < 8),['PlayId','DistToRusherDefHalfNew'] ].groupby('PlayId').count()
    defs_in_9HN = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDefHalfNew < 9),['PlayId','DistToRusherDefHalfNew'] ].groupby('PlayId').count()
    defs_in_10HN = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDefHalfNew < 10),['PlayId','DistToRusherDefHalfNew'] ].groupby('PlayId').count()

    defs_in_colsHN = ['DefsIn1HN','DefsIn2HN','DefsIn3HN','DefsIn4HN','DefsIn5HN','DefsIn6HN','DefsIn7HN','DefsIn8HN','DefsIn9HN','DefsIn10HN']

    defs_in_1HN.columns = [defs_in_colsHN[0]]
    defs_in_2HN.columns = [defs_in_colsHN[1]]
    defs_in_3HN.columns = [defs_in_colsHN[2]]
    defs_in_4HN.columns = [defs_in_colsHN[3]]
    defs_in_5HN.columns = [defs_in_colsHN[4]]
    defs_in_6HN.columns = [defs_in_colsHN[5]]
    defs_in_7HN.columns = [defs_in_colsHN[6]]
    defs_in_8HN.columns = [defs_in_colsHN[7]]
    defs_in_9HN.columns = [defs_in_colsHN[8]]
    defs_in_10HN.columns = [defs_in_colsHN[9]]

    play_train_df = play_train_df.merge(defs_in_1HN, how = 'left', on = 'PlayId') 
    play_train_df = play_train_df.merge(defs_in_2HN, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_3HN, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_4HN, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_5HN, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_6HN, how = 'left', on = 'PlayId') 
    play_train_df = play_train_df.merge(defs_in_7HN, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_8HN, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_9HN, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_10HN, how = 'left', on = 'PlayId')
    

In [ ]:
def pre_process(train_df):         

    # how many defenders within distances NEW
    defs_in_1N = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDefNew < 1),['PlayId','DistToRusherDefNew'] ].groupby('PlayId').count()
    defs_in_2N = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDefNew < 2),['PlayId','DistToRusherDefNew'] ].groupby('PlayId').count()
    defs_in_3N = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDefNew < 3),['PlayId','DistToRusherDefNew'] ].groupby('PlayId').count()
    defs_in_4N = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDefNew < 4),['PlayId','DistToRusherDefNew'] ].groupby('PlayId').count()
    defs_in_5N = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDefNew < 5),['PlayId','DistToRusherDefNew'] ].groupby('PlayId').count()
    defs_in_6N = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDefNew < 6),['PlayId','DistToRusherDefNew'] ].groupby('PlayId').count()
    defs_in_7N = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDefNew < 7),['PlayId','DistToRusherDefNew'] ].groupby('PlayId').count()
    defs_in_8N = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDefNew < 8),['PlayId','DistToRusherDefNew'] ].groupby('PlayId').count()
    defs_in_9N = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDefNew < 9),['PlayId','DistToRusherDefNew'] ].groupby('PlayId').count()
    defs_in_10N = train_df.loc[(~train_df.IsOnOffense) & (train_df.DistToRusherDefNew < 10),['PlayId','DistToRusherDefNew'] ].groupby('PlayId').count()

    defs_in_colsN = ['DefsIn1N','DefsIn2N','DefsIn3N','DefsIn4N','DefsIn5N','DefsIn6N','DefsIn7N','DefsIn8N','DefsIn9N','DefsIn10N']

    defs_in_1N.columns = [defs_in_colsN[0]]
    defs_in_2N.columns = [defs_in_colsN[1]]
    defs_in_3N.columns = [defs_in_colsN[2]]
    defs_in_4N.columns = [defs_in_colsN[3]]
    defs_in_5N.columns = [defs_in_colsN[4]]
    defs_in_6N.columns = [defs_in_colsN[5]]
    defs_in_7N.columns = [defs_in_colsN[6]]
    defs_in_8N.columns = [defs_in_colsN[7]]
    defs_in_9N.columns = [defs_in_colsN[8]]
    defs_in_10N.columns = [defs_in_colsN[9]]

    play_train_df = play_train_df.merge(defs_in_1N, how = 'left', on = 'PlayId') 
    play_train_df = play_train_df.merge(defs_in_2N, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_3N, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_4N, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_5N, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_6N, how = 'left', on = 'PlayId') 
    play_train_df = play_train_df.merge(defs_in_7N, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_8N, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_9N, how = 'left', on = 'PlayId')
    play_train_df = play_train_df.merge(defs_in_10N, how = 'left', on = 'PlayId')    
    

    all_count_cols = defs_in_cols + defs_in_colsN + defs_in_bcols + defs_in_bcolsN + defs_in_bcolsHN + defs_in_colsHN
    for col in all_count_cols:
        play_train_df.loc[play_train_df[col].isnull(),col] = 0.0
        

In [ ]:
def pre_process(train_df):         
    #check nulls
    play_train_df.isnull().sum()

    play_train_df.drop('PlayId', axis = 1, inplace = True)
    
    return play_train_df

In [ ]:
#####
train_df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', low_memory=False)

In [ ]:
#get our output and drop it from the training data
output = train_df[['PlayId','Yards']]
output.drop_duplicates(inplace = True)
output = output['Yards']

train_df.drop('Yards', axis = 1, inplace = True)

yards_lower_bound = 99 #negative 99
output_len = 100 + yards_lower_bound

y = np.zeros((output.shape[0], output_len))
for idx, target in enumerate(list(output)):
    y[idx][yards_lower_bound + target] = 1

In [ ]:
def pre_process(train_df):   
#数据预处理
    play_train_df = pre_process(train_df)

In [ ]:
def pre_process(train_df):   
#standard scale the true numerical variables
    play_train_num_cols = ['Distance','DefendersInTheBox', 'YardsFromOwnGoal',
       'X_std_rush', 'Y_std_rush', 'X_std_new_rush',
       'Y_std_new_rush', 'X_std_half_new_rush', 'Y_std_half_new_rush',
       'S_Rush', 'A_Rush', 'X_Speed_Rush', 'Y_Speed_Rush', 'X_Acc_Rush',
       'Y_Acc_Rush','DistToLOS_Rusher',
       'DistToRusherDefMean', 'DistToRusherDefMin', 'DistToRusherDefMeanNew',
       'DistToRusherDefMinNew', 'DistToRusherDefMeanHalfNew',
       'DistToRusherDefMinHalfNew', 'DefsInb1',
       'DefsInb2', 'DefsInb3', 'DefsInb4', 'DefsInb5', 'DefsInb6', 'DefsInb7',
       'DefsInb8', 'DefsInb9', 'DefsInb1HN', 'DefsInb2HN', 'DefsInb3HN',
       'DefsInb4HN', 'DefsInb5HN', 'DefsInb6HN', 'DefsInb7HN', 'DefsInb8HN',
       'DefsInb9HN', 'DefsInb1N', 'DefsInb2N', 'DefsInb3N', 'DefsInb4N',
       'DefsInb5N', 'DefsInb6N', 'DefsInb7N', 'DefsInb8N', 'DefsInb9N',
       'DefsIn1', 'DefsIn2', 'DefsIn3', 'DefsIn4', 'DefsIn5', 'DefsIn6',
       'DefsIn7', 'DefsIn8', 'DefsIn9', 'DefsIn10', 'DefsIn1HN', 'DefsIn2HN',
       'DefsIn3HN', 'DefsIn4HN', 'DefsIn5HN', 'DefsIn6HN', 'DefsIn7HN',
       'DefsIn8HN', 'DefsIn9HN', 'DefsIn10HN', 'DefsIn1N', 'DefsIn2N',
       'DefsIn3N', 'DefsIn4N', 'DefsIn5N', 'DefsIn6N', 'DefsIn7N', 'DefsIn8N',
       'DefsIn9N', 'DefsIn10N']

    play_train_scaler = StandardScaler()

    play_train_df[play_train_num_cols] = play_train_scaler.fit_transform(play_train_df[play_train_num_cols])

    play_train_arr = play_train_df.values
    play_train_arr.shape

    full_train = play_train_arr

In [ ]:
train = pd.read_csv('../input/nfl-big-data-bowl-2020/train.csv',)

In [ ]:
train.to_csv('l_train.csv',index=False)